In [ ]:
import pandas as pd
import requests
import folium
import numpy as np

In [ ]:
# Create a Map that takes array of [locations, total_reviews] as an input, gets the Geo-Location of each location (using OpenStreeMap API) and Pin-Points the Top-5 Locations with the most reviews

import requests

map = folium.Map(location=[31.786060, 35.200779], zoom_start=7)

df = pd.read_csv("data\\reviews.csv")

df['Location'] = df['Location'].astype(str)
df['Location'] = df['Location'][~df['Location'].str.contains(r'\d')]
places = df['Location'].value_counts().to_dict()

places_dict = []
for location, total_reviews in places.items():
    places_dict.append({'name': location, 'total_reviews': total_reviews})

places_dict= sorted(places_dict, key=lambda x: x['total_reviews'], reverse=True)

for place in places_dict[:5]:
    popup = folium.Popup(f'<center><font size="2"><b>{place["name"]}</b></font><br><u>ביקורות</u>: {place["total_reviews"]}', max_width=300)

    url = f'https://nominatim.openstreetmap.org/search?q={place["name"]}+Israel&format=json'
    response = requests.get(url).json()
    if response:
        lat = response[0]['lat']
        lon = response[0]['lon']

    folium.Marker(location=[lat, lon], popup=popup).add_to(map)

# map

In [ ]:
# Checking Population of each Town in Israel (after getting the Geo-Location of them)
df_towns = pd.read_csv("data\\israel_towns.csv")

map = folium.Map(location=[31.786060, 35.200779], zoom_start=7)

for index, row in df_towns.iterrows():
    if( pd.notnull(row['Geo_lat']) & pd.notnull(row['Geo_lon']) ):
        popup = folium.Popup(f'<center><font size="2"><b>{row["Name"]}</b></font><br><u>תושבים:</u>: {row["Population"]}', max_width=300)

        folium.Marker(location=[row["Geo_lat"], row["Geo_lon"]], popup=popup).add_to(map)

# map